### Import external python files

In [32]:
%matplotlib inline
%run ../skeleton.py
%run ../animation.py

### Load the skeleton definition
This is pickled from maya

In [33]:
skeleton = load_skeleton(r'../skeleton_no_fingers.dat')
animations = [load_animation(r"../motion_graph/anim_{}.dat".format(i)) for i in range(28)]

### Draw Skeleton
We will use k3d to display the skeleton in a view port

In [3]:
import k3d

def plot_skeleton(plot, skeleton, width=0.05):
    for i in range(len(skeleton._bones)):
        if skeleton._bones[i]._parentId >= 0:
            startM = skeleton.globalMatrix(i)
            endM = skeleton.globalMatrix(skeleton._bones[i]._parentId)
            p = k3d.line([startM[3][:3], endM[3][:3]], width=width, color=0x2233FF)
            plot += p

def plot_anchor(plot, skeleton):
    p = k3d.points([
            skeleton.anchorGlobalPosition(i) for i in range(len(skeleton._anchors))
        ], point_size=0.02, color=0x55FF00)
    plot += p
    
def plot_animation(plot, skeleton, animation):
    #plot skeleton at 2 frames
    keycount, tracks = animation
    skeleton.load_animation(animation, int(keycount/2) )
    plot_skeleton(plot, skeleton)

    
    #plot trajectories
    boneIds = [skeleton.bone_id(n) for n in ['Hips','LeftHand','RightHand','LeftFoot','RightFoot']]
    lines = [[] for i in range(len(boneIds))]
    for frame in range(keycount):
        skeleton.load_animation(animation, frame )
        for i,boneId in enumerate(boneIds):
            lines[i].append(skeleton.globalMatrix(boneId)[3][:3])
            
    for i in range(len(boneIds)):
        p = k3d.line(lines[i], shader='simple', color=0xFF22FF)
        plot += p
            
plot = k3d.plot()    
plot_skeleton(plot, skeleton) 
plot_anchor(plot, skeleton)
plot.display()

Output()

In [55]:
import numpy as np
from scipy.spatial import Delaunay

def build_skeleton_edges(skeleton):
    edges = []
    extra_edges = []
    for bone in skeleton._bones[1:]:
        children = skeleton.bone_children(bone._id)
        #first connect all my children to me
        for b in children:
            edges.append([bone._id-1, b._id-1])
        #connect all the children together
        for a in children:
            for b in children:
                if a != b:
                    extra_edges.append([a._id-1, b._id-1])
    return edges, extra_edges

def triangulate_skeleton(skeleton):
    points = [skeleton.globalMatrix(i)[3][:3] for i in range(1, len(skeleton._bones))]
    points += [skeleton.anchorGlobalPosition(i) for i in range(len(skeleton._anchors))]
    points = np.array(points)
    
    tri = Delaunay(points, qhull_options = 'Qt Qbb Qc').simplices
    
    #discard flat tetrahedron
    keep = np.ones(len(tri), dtype = bool)
    for i, t in enumerate(tri):
        if abs(np.linalg.det(np.hstack((points[t], np.ones([1,3+1]).T)))) < 1E-15:
            keep[i] = False # Point is coplanar, we don't want to keep it
    tri = tri[keep]
    
    #keep unique edges only
    edges = np.array([
        flattentup for tup in 
        [[[t[0],t[1]],[t[2],t[0]],[t[1],t[3]],[t[0],t[2]],[t[3],t[1]],[t[2],t[3]]] for t in tri] 
        for flattentup in tup])
    edges.sort()
    edges = np.unique(edges, axis=0)
    
    #keep edges that do not connect parent and child bones
    bone_count = len(skeleton._bones)
    def bone_id(pointId):
        if pointId < bone_count:
            return pointId
        return skeleton._anchors[pointId-bone_count]._parentId
        
    edges_to_keep = []
    for ed in edges:
        a = bone_id(ed[0])
        b = bone_id(ed[1])
        if a != b and skeleton._bones[b]._parentId != a and skeleton._bones[a]._parentId != b:
            edges_to_keep.append(ed)
    edges = edges_to_keep
    
    #keep edges that are not in the skeleton already
    skeleton_edges, skeleton_extra_edges = build_skeleton_edges(skeleton)
    skeleton_edges += skeleton_extra_edges
    
    edges_to_keep = []
    for ed in edges:
        a = bone_id(ed[0])
        b = bone_id(ed[1])
        found = next((True for skl in skeleton_edges if (skl[0]==a and skl[1]==b) or (skl[0]==b and skl[1]==1)),False)
        if found == False:
            edges_to_keep.append(ed)
    edges = edges_to_keep
    
    return points, edges
    

    
def plot_edges(plot, points, edges, color=0xFF3300):
    for edge in edges:
        p = k3d.line([points[edge[0]], points[edge[1]]], color=color)
        plot += p
            

skeleton.load_animation(animations[0], 10 )
points, edges = triangulate_skeleton(skeleton)
skeleton_edges, skeleton_extra_edges = build_skeleton_edges(skeleton)
plot = k3d.plot() 
plot_edges(plot, points, skeleton_extra_edges, 0xFF0000)
plot_edges(plot, points, skeleton_edges, 0xFF3355)
plot_edges(plot, points, edges, 0x0033FF)
plot.display()

Output()

In [48]:
from scipy import sparse

def laplacianUmbrellaMatrix(points, edges):
    """compute a laplacian matrix with umbrella weights"""
    n = len(points)
    I = []
    J = []
    V = []
    
    # Build sparse Laplacian Matrix coordinates and values
    for i in range(n):
        indices = [edge[0] for edge in edges if edge[1] == i]
        indices += [edge[1] for edge in edges if edge[0] == i]
        z = len(indices)
        I = I + ([i] * (z + 1)) # repeated row
        J = J + indices + [i] # column indices and this row
        V = V + ([-1] * z) + [z] # negative weights and row degree
    
    L = sparse.coo_matrix((V, (I, J)), shape=(n, n)).tocsr()
    
    return L

def laplacianInvDistanceMatrix(points, edges):
    """compute a laplacian matrix with umbrella weights"""
    n = len(points)
    I = []
    J = []
    V = []
    
    # Build sparse Laplacian Matrix coordinates and values
    for i in range(n):
        indices = [edge[0] for edge in edges if edge[1] == i]
        distances = [points[edge[0]]-points[edge[1]] for edge in edges if edge[1] == i]
        indices += [edge[1] for edge in edges if edge[0] == i]
        distances += [points[edge[0]]-points[edge[1]] for edge in edges if edge[0] == i]
        distances = np.array(distances)
        #print (distances)
        #raise Exception('stop')
        distances = np.sum(distances*distances, axis=1)
        weights = distances/np.max(distances)* -1
        
        z = len(indices)
        I = I + ([i] * (z + 1)) # repeated row
        J = J + indices + [i] # column indices and this row
        V = V + weights.tolist() + [-np.sum(weights)] # negative weights and row degree
    
    L = sparse.coo_matrix((V, (I, J)), shape=(n, n)).tocsr()
    
    return L

In [56]:
from scipy.optimize import minimize

def _test_laplacian_constraints(skeleton):
    
    points, edges = triangulate_skeleton(skeleton)
    skeleton_edges, skeleton_edges_extra = build_skeleton_edges(skeleton)
    full_skeleton_edges = skeleton_edges + skeleton_edges_extra
    
    def _objective(flattenV):
        #lapacian energy minization
        V = flattenV.reshape((vcount,3))
        lv = L.dot(V) - Lstart
        return np.sum(lv*lv) * 0.5

    def _cnst_bone_length(flattenV, edgeids):
        #constraints the length of the bones
        V = flattenV.reshape((vcount,3))        
        v = np.array( [V[full_skeleton_edges[edgeid][0]]-V[full_skeleton_edges[edgeid][1]] for edgeid in edgeids] )
        b = np.array( [bone_lengths[edgeid] for edgeid in edgeids] )
        return np.sum(b - np.sum(v*v))*.5

    def _cnst_loc_position(flattenV, ptids):
        #constraints a position
        V = flattenV.reshape((vcount,3))
        v = np.array([points[ptid] - V[ptid] for ptid in ptids])
        return np.sum(v*v)
    
    def _bone_lengths(points):
        v = np.array([points[a]-points[b] for a,b in full_skeleton_edges])
        return np.sum(v*v, axis=1)
    
    
    plot = k3d.plot()    
    plot_edges(plot, points, skeleton_edges, 0xFF3300)
    plot.display()
    
    #-----------------------------------------
    bone_lengths = _bone_lengths(points)
    bone_lengths *= 0.5
    
    
    #setup constraints
    #-----------------------------------------
    constraints = []
    
    #bone length
    constraints += [
        {'type':'eq', 'fun':lambda v, indexes=[i]:_cnst_bone_length(v,indexes)} for i in range(len(skeleton_edges))
    ]
    
    #foot contact
    constraints += [
        {'type':'eq', 'fun':lambda v, indexes=[
            (skeleton.bone_id('LeftFootToes') - 1)
        ]:_cnst_loc_position(v,indexes)}
    ]
    
    
    
    
    #Test with umbrella weights
    #--------------------------------------------------------------
    L = laplacianUmbrellaMatrix(points, edges+skeleton_edges_extra+skeleton_edges)
    vcount = len(points)
    Lstart = L.dot(points)
    
    v0 = points.copy()
    for i in range(5):
        sol = minimize(_objective, v0, method='SLSQP', constraints=constraints)
        v0 = sol.x.reshape((vcount,3))
        print (sol.message)
        if sol.success:
            break

    plot_edges(plot, v0, skeleton_edges, 0x0033FF)
    
    #Test with distance weights
    #--------------------------------------------------------------
    L = laplacianInvDistanceMatrix(points, edges+skeleton_edges)
    vcount = len(points)
    Lstart = L.dot(points)
    
    v0 = points.copy()
    for i in range(5):
        sol = minimize(_objective, v0, method='SLSQP', constraints=constraints)
        v0 = sol.x.reshape((vcount,3))
        print (sol.message)
        if sol.success:
            break

    plot_edges(plot, v0, skeleton_edges, 0x005500)
    
    
_test_laplacian_constraints(skeleton)

Output()

Iteration limit exceeded
Iteration limit exceeded
Iteration limit exceeded
Iteration limit exceeded
Optimization terminated successfully.
Iteration limit exceeded
Optimization terminated successfully.


In [25]:
skeleton.bone_id('LeftFootToes')

22